# Investment Strategy Analysis for Inherited Wealth

Welcome to this Jupyter Notebook, designed as part of our assignment to develop an investment strategy for an inherited sum of R100,000. This analysis is aimed at identifying the most promising investment avenues through a mix of broad-based unit trusts and exchange-traded funds (ETFs), specifically avoiding single stocks to align with best practices.

The objective is to utilize quantitative and qualitative analysis methods to evaluate different markets and asset classes. This will include studying historical performances, applying statistical techniques, and forecasting future returns to make informed investment decisions that could potentially maximize the return on the inherited wealth by the time I reach the age of 60.


## Assignment Objectives and Analysis Framework

This notebook is structured to systematically analyze various ETFs and indexes using data from the Alpha Vantage API. Here’s what we aim to achieve:

1. **Data Collection**:
    - Fetch daily time series data, along with technical indicators such as Simple Moving Average (SMA) and Relative Strength Index (RSI) for selected ETFs.
    - Incorporate fundamental data to gain deeper insights into each ETF’s characteristics.

2. **Quantitative Analysis**:
    - Employ descriptive statistics to understand the data’s central tendencies and variability.
    - Use linear regression models to predict future trends based on historical data.

3. **Qualitative Analysis**:
    - Consider external factors such as economic indicators, market conditions, and geopolitical events that could influence the markets.

4. **Investment Recommendations**:
    - Synthesize the quantitative data and qualitative insights to recommend the most suitable markets for investment.
    - Provide a forecast of the potential growth of the investment by the age of 60, illustrating the long-term benefits of the chosen strategy.

5. **Innovative Techniques**:
    - Explore advanced data visualization tools and statistical techniques that go beyond the basic curriculum to enhance the analysis and presentation.

6. **Reflection**:
    - Conclude with a reflection on the learning experience, the insights gained about financial markets, the effectiveness of various data analysis techniques, and personal growth in data analysis and programming skills within the scope of this course.

The aim is to present our findings in a visually engaging manner using storytelling techniques, effective visuals, and clear, concise data representations in a PowerPoint presentation complemented by this detailed Jupyter Notebook analysis.


In [5]:
import requests
import pandas as pd
import io

# Function to fetch ETF data
def fetch_etf_data(symbol, api_key):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}&datatype=csv'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.content.decode('utf-8')
        df = pd.read_csv(io.StringIO(data))
        return df
    else:
        print(f"Failed to fetch data for {symbol}: ", response.status_code)
        return None

# Fetching Simple Moving Average (SMA)
def fetch_sma(symbol, api_key):
    sma_url = f'https://www.alphavantage.co/query?function=SMA&symbol={symbol}&interval=daily&time_period=20&series_type=close&apikey={api_key}&datatype=csv'
    return fetch_data(sma_url)

# Fetching Relative Strength Index (RSI)
def fetch_rsi(symbol, api_key):
    rsi_url = f'https://www.alphavantage.co/query?function=RSI&symbol={symbol}&interval=daily&time_period=14&series_type=close&apikey={api_key}&datatype=csv'
    return fetch_data(rsi_url)

# Fetching ETF Overview (Fundamental Data)
def fetch_etf_overview(symbol, api_key):
    overview_url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}'
    response = requests.get(overview_url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch overview for {symbol}: {response.status_code}")
        return None

# General function to fetch data from API
def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.content.decode('utf-8')
        df = pd.read_csv(io.StringIO(data))
        return df
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

# Your API Key
api_key = 'NAESXONQHLPIJRTA'  # Replace YOUR_API_KEY with your actual API key from Alpha Vantage

# List of ETF symbols
etf_symbols = ['SPY', 'VT', 'EEM', 'GLD', 'ICLN', 'VNQ']

# Dictionary to hold data for each ETF
etf_data = {}

# Fetch data for each ETF
for symbol in etf_symbols:
    print(f"\nFetching data for {symbol}...")
    daily_data = fetch_etf_data(symbol, api_key)
    sma_data = fetch_sma(symbol, api_key)
    rsi_data = fetch_rsi(symbol, api_key)
    overview_data = fetch_etf_overview(symbol, api_key)

    if daily_data is not None:
        etf_data[symbol] = {
            'Daily': daily_data,
            'SMA': sma_data,
            'RSI': rsi_data,
            'Overview': overview_data
        }
        print(f"Data for {symbol} fetched successfully!")
    else:
        print(f"Data for {symbol} not available or failed to fetch.")

# Example of how to access the data
for symbol, datasets in etf_data.items():
    print(f"\n{symbol} Data Preview:")
    if datasets['Daily'] is not None:
        print(datasets['Daily'].head())  # Display the first few rows of the daily data
    if datasets['SMA'] is not None:
        print(datasets['SMA'].head())  # Display the first few rows of the SMA data
    if datasets['RSI'] is not None:
        print(datasets['RSI'].head())  # Display the first few rows of the RSI data
    if datasets['Overview'] is not None:
        print(datasets['Overview'])  # Display the overview data



Fetching data for SPY...
Data for SPY fetched successfully!

Fetching data for VT...
Data for VT fetched successfully!

Fetching data for EEM...
Data for EEM fetched successfully!

Fetching data for GLD...
Data for GLD fetched successfully!

Fetching data for ICLN...
Data for ICLN fetched successfully!

Fetching data for VNQ...
Data for VNQ fetched successfully!

SPY Data Preview:
    timestamp    open    high      low   close    volume
0  2024-05-03  511.16  512.55  508.560  511.29  72756709
1  2024-05-02  504.15  505.89  499.550  505.03  62550179
2  2024-05-01  501.38  508.19  499.865  500.35  80242839
3  2024-04-30  508.56  509.56  501.980  501.98  77483566
4  2024-04-29  510.09  510.75  507.250  510.06  46415449
         time      SMA
0  2024-05-03  506.773
1  2024-05-02  507.130
2  2024-05-01  507.532
3  2024-04-30  508.485
4  2024-04-29  509.328
         time      RSI
0  2024-05-03  53.8995
1  2024-05-02  47.3777
2  2024-05-01  41.6470
3  2024-04-30  43.1674
4  2024-04-29  51.88

In [9]:
import pandas as pd
import numpy as np

def clean_etf_data(df):
    """
    Cleans ETF data by handling missing values, correcting data types, and removing duplicates.
    Args:
    df (DataFrame): The DataFrame containing ETF data.
    Returns:
    DataFrame: The cleaned DataFrame.
    """
    # Convert date column to datetime format
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    
    # Sort data by date
    df.sort_values('timestamp', inplace=True)
    
    # Remove any duplicates
    df.drop_duplicates(subset=['timestamp'], keep='last', inplace=True)
    
    # Fill missing numeric values with the mean
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    for col in numeric_cols:
        df[col].fillna(df[col].mean(), inplace=True)
    
    # Check for any remaining null values and drop if necessary
    if df.isnull().sum().sum() > 0:
        df.dropna(inplace=True)
        
    return df

def calculate_moving_averages(df, window=20):
    """
    Calculates the moving average of the 'close' price.
    Args:
    df (DataFrame): The DataFrame with stock data.
    window (int): The window size to calculate the moving average.
    Returns:
    Series: A pandas Series containing the moving average of the closing prices.
    """
    return df['close'].rolling(window=window).mean()

def detect_outliers(df, n_std=3):
    """
    Identifies outliers in the 'close' price column based on standard deviation.
    Args:
    df (DataFrame): The DataFrame to analyze.
    n_std (int): The number of standard deviations to use as a threshold.
    Returns:
    DataFrame: DataFrame with an additional column 'Outlier' marking outliers as True.
    """
    mean = df['close'].mean()
    std = df['close'].std()
    
    df['Outlier'] = (df['close'] > mean + n_std * std) | (df['close'] < mean - n_std * std)
    return df

# Example usage
# Assume 'etf_data' is your DataFrame loaded with data from Alpha Vantage
etf_data_cleaned = clean_etf_data(etf_data)
moving_averages = calculate_moving_averages(etf_data_cleaned)
etf_data_with_outliers = detect_outliers(etf_data_cleaned)

print(etf_data_cleaned.head())
print(moving_averages.head())
print(etf_data_with_outliers[etf_data_with_outliers['Outlier']])


KeyError: 'timestamp'